In [1]:
# 240626

from anthropic import AnthropicVertex
import os
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor
from itertools import product
import json
from util_trans import *

REGION = 'us-east5'
PROJECT_ID = "phonic-impact-421908"
ENDPOINT = f"https://{REGION}-aiplatform.googleapis.com"

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
client = AnthropicVertex(region=REGION, project_id=PROJECT_ID)

In [2]:
class ModelResponder:
    def __init__(self, model_path, ques_path_list, context_list, prompt_func_list=None, path=None):
        self.batch_size = 10
        self.model_path = model_path
        if path is not None:
            self.path = path
        else:
            self.path = os.path.basename(model_path)

        if not prompt_func_list:
            default_prompt_func = lambda inst,ques: f"{ques}"
            prompt_func_list = [default_prompt_func]
        self.prompt_func_list = prompt_func_list

        indexed_ques_paths = list(enumerate(ques_path_list))
        indexed_prompt_funcs = list(enumerate(self.prompt_func_list))
        indexed_contexts = list(enumerate(context_list))
        self.combinations = list(product(indexed_ques_paths, indexed_prompt_funcs, indexed_contexts))

    def process_and_update(self, ques_dict, context, prompt_func, pbar):
        max_retries = 50
        retries = 0
        while retries < max_retries:
            try:
                responses = client.messages.create(
                    messages=[{"role":"user","content": prompt_func(inst, ques_dict)}],
                    system=context,
                    model=model_path,
                    temperature=0,
                    max_tokens=1024
                )
                pbar.update(1)
                output = responses.to_dict()['content'][0]['text']
                try: result = ast.literal_eval(extract_substring(output))
                except: result = output_adjust(extract_substring(output))
                return result
            except Exception as e:
                if "Quota" in str(e):
                    time.sleep(60)
                elif "he candidate is likely blocked" in str(e):
                    retries += 50
                    print(f'Candidate blocked, skipping... {ques_dict['year']}-{ques_dict['session']}-{ques_dict['question_number']}"')
                else:
                    retries += 1
                    print(e)
                    print(f"Retrying '{ques_dict['year']}-{ques_dict['session']}-{ques_dict['question_number']}'... ({retries}/{max_retries})")
                    print(output)
                    print(result)
                    time.sleep(10)

    def process_files(self):
        for (ques_idx, ques_path), (prompt_idx, prompt_func), (context_idx, inst) in self.combinations:
            with open(ques_path) as f:
                exam = json.load(f)
            filename=f'output/{self.path} [f{prompt_idx+1}_p{context_idx+1}_q{ques_idx+1}].json'
            dataset = ExamDataset(exam, inst, prompt_func)
            dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
            with tqdm(total=len(dataloader),desc=filename, leave=True, position=0) as pbar:
                for i, batch in enumerate(dataloader):
                    with ThreadPoolExecutor() as executor:
                        with tqdm(total=len(batch), desc=f"Batch {i}", leave=True, position=0) as batch_pbar:
                            results = list(executor.map(lambda ques: self.process_and_update(ques, inst, prompt_func, batch_pbar), batch))
                    if os.path.exists(filename):
                        with open(filename, 'r') as file:
                            resp = json.load(file)
                    else:
                        resp = []
                        os.makedirs(os.path.dirname(filename), exist_ok=True)
                    for result in results:
                        resp.append(result)
                    with open(filename, 'w', encoding='utf-8') as f:
                        json.dump(resp, f, indent=4, ensure_ascii=False)
                    pbar.update(1)

In [3]:
model_path = "claude-3-5-sonnet@20240620"
run = ModelResponder(model_path, ques_path_list, inst_list)
run.process_files()

Batch 110: 100%|██████████| 10/10 [00:03<00:00,  2.54it/s]███▏    | 110/210 [06:59<05:35,  3.35s/it]
output/claude-3-5-sonnet@20240620 [f1_p1_q1].json:  52%|█████▏    | 110/210 [07:03<06:24,  3.85s/it]


invalid decimal literal (<unknown>, line 1)
Retrying '2022-1-60'... (1/50)
Here's the translation of the question into English while maintaining its original format:

{'year': 2022, 'session': 1, 'common_question_text': 'Read the following case and choose the most appropriate answer for each question.', 'common_data': 'The following figure shows the results of testing the contraction of vascular smooth muscle in vitro after isolating it from a white rat's blood vessel.\nThe effects of drug B and drug C on the vascular smooth muscle contraction response of A (epinephrine) are represented as dose-response curves. (Note: '10X' indicates a 10-fold concentration)\n<Data (confidential)>', 'question_number': 60, 'question_text': 'Which of the following correctly describes the action characteristics of C?', 'data': None, 'choices': ['① It shows an additive effect with A.', '② It shows a synergistic effect with A.', '③ It competitively antagonizes A.', '④ It non-competitively antagonizes A.', '

UnboundLocalError: cannot access local variable 'result' where it is not associated with a value

In [ ]:
# model_path = "claude-3-opus@20240229"
# run = ModelResponder(model_path, ques_path_list, inst_list)
# run.process_files()